# Add Date Dimension to your lakehouse 

In [8]:
from pyspark.sql.functions import explode, sequence, to_date
from pyspark.sql.functions import col, year, month, dayofmonth, weekofyear, date_format, to_date, expr

StatementMeta(, 9d32c3b2-d536-46ec-a835-05cd1ed4e8fa, 10, Finished, Available, Finished)

In [9]:
# Parameter
beginDate = '2013-01-01'
endDate = '2014-12-31'
display_data = False

StatementMeta(, 9d32c3b2-d536-46ec-a835-05cd1ed4e8fa, 11, Finished, Available, Finished)

In [14]:
df = spark.sql("SELECT * FROM Excel.calendar LIMIT 1000")
display(df)

StatementMeta(, 9d32c3b2-d536-46ec-a835-05cd1ed4e8fa, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b9417362-ec7a-46df-80f7-ab3467d462f0)

In [10]:
df = spark.sql(f"select explode(sequence(to_date('{beginDate}'), to_date('{endDate}'), interval 1 day)) as date")

StatementMeta(, 9d32c3b2-d536-46ec-a835-05cd1ed4e8fa, 12, Finished, Available, Finished)

In [11]:
date_df = df.select(
    date_format("date","yyyyMMdd").alias("DateKey"),
    date_format("date","yyyy-MM-dd").alias("Date"),
    col("date").alias("Date2Key"),
    year("date").alias("Year"),
    month("date").alias("Month"),
    dayofmonth("date").alias("Day"),
    weekofyear("date").alias("WeekOfYear"),
    date_format("date","yyyy-MM").alias("YearMonth"),
    date_format("date", "E").alias("DayOfWeek")
)

date_df = date_df.createOrReplaceTempView('calendar_temp')

query = """
    SELECT 
        *
        ,DAYOFWEEK(date) AS DayOfWeekNum
        ,CASE WHEN ( YEAR(date) = YEAR(CURRENT_DATE()) ) THEN 1 ELSE 0 END  AS IsCurrentYear
        ,CASE WHEN ( YEAR(date) = YEAR(CURRENT_DATE())-1 ) THEN 1 ELSE 0 END  AS IsPreviousYear
        ,CASE WHEN ( YEAR(date) = YEAR(CURRENT_DATE()) AND QUARTER(date) = QUARTER(CURRENT_DATE()) ) THEN 1 ELSE 0 END  AS IsCurrentQuarter
        ,CASE WHEN ( YEAR(date) = YEAR(CURRENT_DATE()) ) THEN 1 ELSE 0 END  AS IsCurrentMonth
        ,CASE WHEN ( DATE_FORMAT(date, 'yyyy-MM') = DATE_FORMAT(ADD_MONTHS(CURRENT_DATE(), -1), 'yyyy-MM') ) THEN 1 ELSE 0 END  AS IsPreviousMonth
        ,CASE WHEN ( date BETWEEN DATE_ADD(CURRENT_DATE(), -14) AND CURRENT_DATE() ) THEN 1 ELSE 0 END  AS IsInLast14Days
        ,CASE WHEN ( date BETWEEN DATE_ADD(CURRENT_DATE(), -30) AND CURRENT_DATE() ) THEN 1 ELSE 0 END  AS IsInLast30Days
    FROM calendar_temp
"""

final_date_df = spark.sql(query)


StatementMeta(, 9d32c3b2-d536-46ec-a835-05cd1ed4e8fa, 13, Finished, Available, Finished)

In [12]:
if display_data:
    display(final_date_df)

StatementMeta(, 9d32c3b2-d536-46ec-a835-05cd1ed4e8fa, 14, Finished, Available, Finished)

In [13]:
# Write the DataFrame to the lakehouse
final_date_df.write.mode("overwrite").option("mergeSchema", "true").format("delta").saveAsTable("calendar")

StatementMeta(, 9d32c3b2-d536-46ec-a835-05cd1ed4e8fa, 15, Finished, Available, Finished)